In [ ]:
import pandas as pd
import numpy as np

In [ ]:
faults = pd.read_csv('./data/J1939Faults.csv',
                    usecols = ['RecordID',
                               'ESS_Id',
                               'EventTimeStamp',
                               'eventDescription',
                               'ecuSoftwareVersion',
                               'ecuSerialNumber',
                               'ecuModel',
                               'ecuMake',
                               'ecuSource',
                               'spn',
                               'fmi',
                               'active',
                               'activeTransitionCount',
                               'MCTNumber',
                               'Latitude',
                               'Longitude',
                               'LocationTimeStamp'],
                    dtype = {'spn':'object', 'fmi':'object'})
fault_codes = pd.read_excel('./data/Service Fault Codes_1_0_0_167.xlsx',
                           usecols = ['SPN',
                                      'J1939 FMI',
                                      'Lamp Color',
                                      'Lamp Device',
                                      'Cummins Description',
                                      'Algorithm Description'],
                           dtype = 'object')
onboard = pd.read_csv('./data/VehicleDiagnosticOnboardData.csv')


In [ ]:
fault_codes_2 = fault_codes.dropna()

In [ ]:
onboard_faults_make_codes = (
    onboard.\
    pivot(index = 'FaultId', columns = 'Name', values = 'Value').\
    reset_index().\
    merge(faults, left_on = 'FaultId', right_on = 'RecordID').\
    merge(fault_codes_2, left_on = ['spn','fmi'], right_on = ['SPN','J1939 FMI'], how = 'left')
)

In [ ]:
float_list = ['AcceleratorPedal', 
              'BarometricPressure', 
              'CruiseControlSetSpeed',
              'DistanceLtd',
              'EngineCoolantTemperature',
              'EngineOilPressure',
              'EngineOilTemperature',
              'EngineRpm',
              'EngineTimeLtd',
              'FuelLevel',
              'FuelLtd',
              'FuelRate',
              'FuelTemperature',
              'IntakeManifoldTemperature',
              'ServiceDistance',
              'Speed',
              'SwitchedBatteryVoltage',
              'Throttle',
              'TurboBoostPressure']
int_list = ['EngineLoad', 'LampStatus']
bool_list = ['CruiseControlActive', 
             'IgnStatus',
             'ParkingBrake']

In [ ]:
for i in float_list:
    onboard_faults_make_codes[i] = (
        onboard_faults_make_codes[i].str.replace(',','.').fillna(0).astype(float)
    )

for i in int_list:
    onboard_faults_make_codes[i] = (
        onboard_faults_make_codes[i].fillna(0).astype(int)
    )

for i in bool_list:
    onboard_faults_make_codes[i] = (
        onboard_faults_make_codes[i].fillna(False).astype(bool)
    )
